# Construcción de Blockchain con Python

Diego Rodríguez García. 15/10/2023.

## 1. Ajustes

### `1.1. Instalaciones`

In [1]:
!pip install flask-ngrok==0.0.25

### `1.2. Librerías`

In [2]:
import datetime
import hashlib
import json
from flask import Flask, jsonify
from flask_ngrok import run_with_ngrok

## 2. Clase

Creación de una clase Blockchain que contenga todos los métodos esenciales, como son los siguientes:     
* Creación de un nuevo bloque
* Obtención del hash de un bloque
* Protocolo de concenso Proof of Work (PoW)
* Generación del hash de un bloque
* Verificación de la validez de la Blockchain

In [3]:
class Blockchain:

  #Constructor de la clase Blockchain. 
  def __init__(self):
    self.chain = []
    self.create_block(proof = 1, previous_hash = '0')
      
  #Creación de un nuevo bloque. Argumentos proof (nounce del bloque actual) y previous_hash (hash del bloque previo).
  def create_block(self, proof, previous_hash):
    block = { 'index'         : len(self.chain)+1,
              'timestamp'     : str(datetime.datetime.now()),
              'proof'         : proof,
              'previous_hash' : previous_hash}
    self.chain.append(block)

    #Devolvemos el bloque creado
    return block

  #Obtención del bloque anterior.
  def get_previous_block(self):
    return self.chain[-1]
  
  #Protocolo de concenso Proof of Work (PoW). Argumento previous_proof (nounce del bloque anterior).
  def proof_of_work(self, previous_proof):
    new_proof = 1
    check_proof = False
    while check_proof is False:
        hash_operation = hashlib.sha256(str(new_proof**2 - previous_proof**2).encode()).hexdigest()
        if hash_operation[:4] == '0000':
            check_proof = True
        else: 
            new_proof += 1

    #Devolvemos el nuevo nounce.
    return new_proof
  
  #Cálculo del hash de un bloque.
  def hash(self, block):
    encoded_block = json.dumps(block, sort_keys = True).encode()
    hash_block = hashlib.sha256(encoded_block).hexdigest()
    return hash_block
  
  #Determinación de validez de blockchain.
  def is_chain_valid(self, chain):
    previous_block = chain[0]
    block_index = 1
    while block_index < len(chain):
        block = chain[block_index]
        if block['previous_hash'] != self.hash(previous_block):
            return False
        previous_proof = previous_block['proof']
        proof = block['proof']
        hash_operation = hashlib.sha256(str(proof**2 - previous_proof**2).encode()).hexdigest()
        if hash_operation[:4] != '0000':
            return False
        previous_block = block
        block_index += 1
    return True

## 3. Aplicación

Aplicación Web accesible vía en modo de REST API. Mediante llamadas HTTP al REST API podemos establecer uan comunicación con el uso del módulo de Flask. 

Llamadas a realizar vía REST API:
* Minación de bloques: **mine_block()**
* Obtención de la Blockchain: **get_chain()**
* Comprobar estado de la Blockchain: **is_valid()**

In [4]:
#Crear una aplicación web

#Ejecución de la app con Notebook
app = Flask(__name__)
run_with_ngrok(app)  

#Si se obtiene un error 500, actualizar flask y ejecutar la siguiente línea
app.config['JSONIFY_PRETTYPRINT_REGULAR'] = False

#Creación de la Blockchain
blockchain = Blockchain()


@app.route('/mine_block', methods=['GET'])
def mine_block():
  """ Minado de un nuevo bloque """

  previous_block  = blockchain.get_previous_block()
  previous_proof  = previous_block['proof']
  proof           = blockchain.proof_of_work(previous_proof)
  previous_hash   = blockchain.hash(previous_block)
  block           = blockchain.create_block(proof, previous_hash)
  response = {'message'       : '¡Enhorabuena, has minado un nuevo bloque!', 
              'index'         : block['index'],
              'timestamp'     : block['timestamp'],
              'proof'         : block['proof'],
              'previous_hash' : block['previous_hash']}
  return jsonify(response), 200

@app.route('/get_chain', methods=['GET'])
def get_chain():
  """ Obtención de la Blockchain """
  response = {'chain'   : blockchain.chain, 
              'length'  : len(blockchain.chain)}
  return jsonify(response), 200

@app.route('/is_valid', methods = ['GET'])
def is_valid():
  """ Comprobación si la Blockchain es válida """

  is_valid = blockchain.is_chain_valid(blockchain.chain)
  if is_valid:
      response = {'message' : '¡La cadena de bloques es válida!'}
  else:
      response = {'message' : '¡La cadena de bloques NO es válida!'}
  return jsonify(response), 200  

## 4. Llamada

In [5]:
app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit


 * Running on http://dd54-80-26-196-216.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [15/Oct/2023 13:12:06] "GET / HTTP/1.1" 404 -
127.0.0.1 - - [15/Oct/2023 13:12:46] "GET /mine_block HTTP/1.1" 200 -
127.0.0.1 - - [15/Oct/2023 13:13:32] "GET /mine_block HTTP/1.1" 200 -
127.0.0.1 - - [15/Oct/2023 13:13:34] "GET /mine_block HTTP/1.1" 200 -
127.0.0.1 - - [15/Oct/2023 13:13:35] "GET /mine_block HTTP/1.1" 200 -
127.0.0.1 - - [15/Oct/2023 13:13:37] "GET /mine_block HTTP/1.1" 200 -
127.0.0.1 - - [15/Oct/2023 13:13:39] "GET /mine_block HTTP/1.1" 200 -
127.0.0.1 - - [15/Oct/2023 13:13:41] "GET /mine_block HTTP/1.1" 200 -
127.0.0.1 - - [15/Oct/2023 13:13:43] "GET /mine_block HTTP/1.1" 200 -
127.0.0.1 - - [15/Oct/2023 13:13:44] "GET /mine_block HTTP/1.1" 200 -
127.0.0.1 - - [15/Oct/2023 13:13:58] "GET /get_chain HTTP/1.1" 200 -
127.0.0.1 - - [15/Oct/2023 13:15:03] "GET /mine_block HTTP/1.1" 200 -
127.0.0.1 - - [15/Oct/2023 13:15:09] "GET /mine_block HTTP/1.1" 200 -
127.0.0.1 - - [15/Oct/2023 13:15:12] "GET /mine_block HTTP/1.1" 200 -
127.0.0.1 - - [15/Oct/2023 13:1